# Documentação
> Zendesk Chat

- Atualmente o processo carrega apenas a tabela "chats".

1) [Chat-API - Export Full](https://developer.zendesk.com/api-reference/live-chat/chat-api/chats/)
2) [Chat-API - Export Incremental](https://developer.zendesk.com/api-reference/live-chat/chat-api/incremental_export/)

# Processo de Carga
> Processo completo de ETL.

In [ ]:
%run /Utils/Functions/core

In [ ]:
%sql
use catalog prod;

## Bibliotecas
> Import das bibliotecas utilizadas no processo.

In [ ]:
import requests
import json
from datetime import datetime, timedelta
from time import sleep, mktime
from uuid import uuid4
import concurrent.futures

from IPython.display import clear_output

In [ ]:
## Cria as variáveis de LOG
format_log = '%Y-%m-%d %H:%M:%S'
dtInicio = datetime.today() - timedelta(hours=3)
dtInicio_format = dtInicio.strftime(format_log)
tipo_log = 'API'
camada = 'landing'
emissor = '<org>'
atividade = '<activity_desc>'
origem = 'RestService'
destino = 'AzureBlobFS'
execUrl = ' '

try:
    infos = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson()) # captura as informações do job que executa o notebook
    orgId = infos['tags']['orgId']
    runId = infos['tags']['multitaskParentRunId']
    jobId = infos['tags']['jobId']
    if orgId == '2960871991268730': # Monta a URL caso seja o ID do ambiente de DEV
        execUrl = f'https://adb-{orgId}.10.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
    else: # Monta a URL caso seja o ID do ambiente de PROD
        execUrl = f'https://adb-{orgId}.15.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
except:
    print('Campo de URL não pode ser identificado!')

## Parâmetros
> Lê os parâmetros de entrada do notebook.
> Define valores default.

In [ ]:
dbutils.widgets.text("dt_ingestao", "")

## Variáveis
> Definição e inicialização das variáveis utiliadas no processo.

In [ ]:
location_landing = spark.sql("show external locations").select("url").where("name = 'landing-area'").collect()[0][0]

dt_ingestao = getArgument("dt_ingestao").upper().strip()

# Armazena o token de autenticação da API
auth_token = dbutils.secrets.get("scope-vault-data", "zendesk-chat-token")

# Formata o dt_ingestao
format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
dt_ingestao = datetime.now() if dt_ingestao == "" else datetime.strptime(dt_ingestao, format_timestamp)

# Pega o horário atual e tira 3 horas para converter para BRT (UTC -3)
# Tira 5 minutos por conta de um limite da API
dt_fim = dt_ingestao - timedelta(hours=3) - timedelta(minutes=5)
dt_inicio = dt_fim - timedelta(days=1)

# Força as horas, minutos e segundos em 0
dt_inicio = datetime(dt_inicio.year, dt_inicio.month, dt_inicio.day, 0, 0, 0)

# Separa as datas no formato UNIX
dt_fim_unix = int(mktime(dt_fim.timetuple()))
dt_inicio_unix = int(mktime(dt_inicio.timetuple()))

# Transforma as datas em STRING utilizando um formato de timestamp
dt_fim = dt_fim.strftime(format_timestamp)
dt_inicio = dt_inicio.strftime(format_timestamp)

# Qtd de Threads em paralelas
threads = 10

print(f"dt_inicio: {dt_inicio} | unix: {dt_inicio_unix}")
print(f"dt_fim   : {dt_fim} | unix: {dt_fim_unix}")

### Tabelas
> Lê a tabela de parâmetros para pegar as cargas ativas a serem processadas.

In [ ]:
df_param = fn_ConsultaJdbc("""
    SELECT pca.*
    FROM ctl.ADF_Parametro_Carga_API pca
    WHERE pca.fl_ativo = 1
    and nm_Sistema = 'zendesk_chat'
""")
data_param = df_param.collect()

display(df_param)

## Configurações
> Define as configurações do processo de carga.

In [ ]:
baseUrl = "https://www.zopim.com"
headers = {'Authorization': auth_token, 'Content-Type': 'application/json'}

## Ingestão
> Processo de ingestão dos dados. Da origem até o datalake.

### Extração dos dados
> Realiza consultas na API e salva os resultados (json) em uma lista.

In [ ]:
def get_tables(param, baseUrl, dt_inicio_unix, dtIngestao, location_landing, headers):
    end_point_page = None
    try:
        format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
        dtIngestao = str(dtIngestao)
        if param.vl_Ultimo_Incremento is not None:
            vlUltimoIncremento = datetime.strptime(param.vl_Ultimo_Incremento, format_timestamp)
            vlUltimoIncremento = int(mktime(vlUltimoIncremento.timetuple()))
        else:
            vlUltimoIncremento = dt_inicio_unix

        vlUltimoIncremento = str(vlUltimoIncremento)

        # Landing
        if param.vl_Schedule_Carga != 0:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
        else:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()

        # URL para o request
        url = baseUrl + param.ds_Url.replace("@DT_INI_CARGA", vlUltimoIncremento)
        
        # Cria uma lista vazia para armazenar as respostas de cada request
        result_data = []
        paginas = 0

        # limite de registros por resposta da api (valor fixo)
        # esse valor deve ser definido de acordo com a documentação da API
        api_limit = 1000

        # número máximo de erros permitodos antes de parar a execução (tentativas)
        max_errors = 10

        # tempo para esperar para tentar novamente em caso de erro (em segundos)
        error_sleep = 30

        # Faz o primeiro Request na API
        response = requests.get(url=url, headers=headers)

        # Verifica o status do request
        if response.status_code == 200:
            print("Primeiro request feito com sucesso.")
            data = response.json()
            result_data.extend(data[param.nm_Item_Origem])

            paginas += 1
            while data["count"] == api_limit or response.status_code != 200:
                if response.status_code != 200:
                    print("Erro no request, esperando para tentar novamente...")
                    max_errors -= 1
                    if(max_errors < 0):
                        raise Exception(str(response.content))

                    sleep(error_sleep)
                else:
                    # print("Novas páginas...")

                    # Atualiza o URL para a próxima página
                    if data["count"] == api_limit:
                        url = data["next_page"]
                    else:
                        print("Processo finalizado com sucesso.")
                        break

                response = requests.get(url=url, headers=headers)

                if response.status_code == 200:
                    data = response.json()
                    result_data.extend(data[param.nm_Item_Origem])

                    if len(result_data) == 0: # caso a quantidade de registros seja 0 (vazio), encerra a execução para evitar erros no LOG
                        return

                    paginas += 1

                    clear_output(wait=True)
                    print(f"Qtd de páginas processadas: {paginas}\nTentativas restantes: {max_errors}")
                else:
                    print(str(response.content))
        else:
            raise Exception(str(response.content))

        fn_SaveJson(result_data, dir_landing, str(param.ds_Nome_Arquivo_Landing).lower())
        fn_AtualizaUltimoIncremento_API(param.id_Parametro_Carga_API, dtIngestao)
        
        ## LOG SUCESSO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "execUrl": execUrl}

        fn_LogSucceeded(parametros, dt_ingestao.strftime(format_log))

        return f"Carga da tabela {param.ds_Nome_Arquivo_Landing} finalizada com sucesso."
    
    except Exception as error:
        ## LOG ERRO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '

        if hasattr(error, 'code'): # captura o código do erro, caso possua o atributo 'code'
            error_code = error.code
        else:
            error_code = 'NULL'
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "cd_erro": error_code, "erro": str(error), "execUrl": execUrl}

        fn_LogFailed(parametros, dt_ingestao.strftime(format_log))

### Execução das Cargas
> Inicia o processo de carga com paralelismo entre threads para as tabelas.

In [ ]:
# Cria uma lista para armazenar todas as tarefas
tasks = []

# Cria uma instância do ThreadPoolExecutor com threads definidas (max_workers)
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    # Percorre todas as pastas do diretório de origem
    for row in data_param:
        # Executa a função fn_StreamFromFolder_csv em uma thread do ThreadPoolExecutor
        task = executor.submit(get_tables, *(row, baseUrl, dt_inicio_unix, dt_fim, location_landing, headers))
                               
        # Adiciona a tarefa à lista de tarefas
        tasks.append(task)

# Aguarda a conclusão de todas as tarefas
_ = concurrent.futures.wait(tasks, return_when='ALL_COMPLETED')

In [ ]:
for task in tasks:
    try:
        print(task.result(),'\n')
    except Exception as error:
        print(error)
        pass